In [ ]:
!nvidia-smi

In [ ]:
!nvidia-smi -L

In [ ]:
import os


if not os.path.exists("helper_functions.py"):
    !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
else:
    print("[INFO] 'helper_functions.py' already exists, skipping download.")

In [ ]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

In [ ]:
import tensorflow_datasets as tfds
#TFDS stands for tensorflow datasets

In [ ]:
dataset_lists=tfds.list_builders()   # list all the datasets
print("food101" in dataset_lists)

In [ ]:
(train_data,test_data),ds_info=tfds.load(name='food101',
                                         split=['train','validation'],
                                         shuffle_files=True,
                                         as_supervised=True,
                                         with_info=True)


#with_info means to Download the metadata also -> This is the ds_info
#shuffle_files state that files needs to shuffled during loading
#as_supervised means that the we will get data in form of tuples (data,label)

In [ ]:
ds_info.features

In [ ]:
class_names=df_info.features['label'].class_name
class_names

In [ ]:
take_one_sample=train_data.take(1)

In [ ]:
for image,label in take_one_sample:
  print(f"""
  Image shape: {image.shape},
  Image dtype: {image.dtype},
  Target_class from label: {label},
  Class name (str_form): {class_names[label.numpy()]}
        """)



In [ ]:
import tensorflow as tf
tf.reduce_min(image),tf.reduce_max(image)

In [ ]:
# To verify the label is associated with right image
import matplotlib.pyplot as plt
plt.imshow(image)
plt.title(class_names[label.numpy()])

plt.axis(False)

# Neural Networks perform the best when data is in a certain way.(batched, normalized)

#However, not all data(including  data from tensorflow datasets comes like this)



#Since we are going to be using EfficientNetBX pretrained model then we don't need to rescale the data. These architectures have rescaling built in

In [ ]:
def preprocess_img(image,label,img_shape=224):
  img=tf.image.resize(image,[img_shape,img_shape])
  img=img/255.
  return tf.cast(img,tf.float32),label

In [ ]:
preprocess_img(image,label)

#Batch and Prepare Datasets

In [ ]:
train_data=train_data.map(map_func=preprocess_img,num_parallel_calls=tf.data.AUTOTUNE)
train_data=train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

test_data=test_data.map(map_func=preprocess_img,num_parallel_calls=tf.data.AUTOTUNE)
test_data=test_data.batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
train_data,test_data

In [ ]:
from helper_functions import create_tensorboard_callback
checkpoint_path="model_checkpoints/cp.ckpt"
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                       save_weights_only=True,
                                                       save_best_only=True,
                                                       monitor='val_accuracy',
                                                       verbose=0)

checkpoint_callback

###Tensorboard callbacks- to log training results
###ModelCheckpoint callbacks- To save our model's progress after feature_extraction

Mixed precision utilizes the combination of float16 and float32 data types to increase the model_performance

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [ ]:
mixed_precision.global_policy()

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation

In [ ]:
import tensorflow as tf
input_shape=(224,224,3)
base_model=tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable=False

inputs=layers.Input(shape=input_shape,name='input_layer')
x=base_model(inputs,training=False)
x=layers.GlobalAveragePooling2D(name='global_average_pooling_layer')(x)
x=layers.Dense(len(class_names))(x) # output layers
outputs=layers.Activation('softmax',dtype=tf.float32,name='softmax_float32')(x)

model=tf.keras.Model(inputs,outputs)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
for layers in model.layers:
  print(layer.name,layer.trainable,layer.dtype,layer,dtype_policy)

In [ ]:
for layer in model.layers[1].layers[:20]:
  print(layer.name,layer.trainable,layer.dtype,layer.dtype_policy)

In [ ]:
tf.get_logger().setLevel("ERROR")
history_101_food_classes_feature_extract=model.fit(train_data,
                                                   epochs=3,
                                                   steps_per_epoch=len(train_data),
                                                   validation_steps=int(0.15*len(test_data)),
                                                   validation_data=test_data,
                                                   callbacks=[create_tensorboard_callback(dir_name='training_logs',
                                                                                          experiment_name='efficientnetb0_101_classes_all_data_feature_extract'),
                                                              checkpoint_callback])

In [ ]:
# Evaluate model (unsaved version) on whole test dataset
results_feature_extract_model = model.evaluate(test_data)
results_feature_extract_model

In [ ]:
# 1. Create a function to recreate the original model
def create_model():
  # Create base model
  input_shape = (224, 224, 3)
  base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False)
  base_model.trainable = False # freeze base model layers

  # Create Functional model
  inputs = layers.Input(shape=input_shape, name="input_layer")
  # Note: EfficientNetBX models have rescaling built-in but if your model didn't you could have a layer like below
  # x = layers.Rescaling(1./255)(x)
  x = base_model(inputs, training=False) # set base_model to inference mode only
  x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
  x = layers.Dense(len(class_names))(x) # want one output neuron per class
  # Separate activation of output layer so we can output float32 activations
  outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x)
  model = tf.keras.Model(inputs, outputs)

  return model

In [ ]:
# 2. Create and compile a new version of the original model (new weights)
created_model = create_model()


# 3. Load the saved weights
created_model.load_weights(checkpoint_path)

created_model.compile(loss="sparse_categorical_crossentropy",
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=["accuracy"])

# 4. Evaluate the model with loaded weights
results_created_model_with_loaded_weights = created_model.evaluate(test_data)

In [ ]:
# Save model locally (if you're using Google Colab, your saved model will Colab instance terminates)
save_dir = "07_efficientnetb0_feature_extract_model_mixed_precision"
model.save(save_dir)

In [ ]:
loaded_saved_model = tf.keras.models.load_model(save_dir)

In [ ]:
results_loaded_saved_model = loaded_saved_model.evaluate(test_data)
results_loaded_saved_model

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_feature_extract_model_mixed_precision.zip

In [ ]:
!mkdir downloaded_gs_model # create new dir to store downloaded feature extraction model
!unzip 07_efficientnetb0_feature_extract_model_mixed_precision.zip -d downloaded_gs_model

In [ ]:
loaded_gs_model=tf.keras.models.load_model("downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision")

In [ ]:
loaded_gs_model.summary()

In [ ]:
earlystopping=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                               patience=3)

checkpoint_path = "fine_tune_checkpoints/"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only=True,
                                                      monitor="val_loss")

In [ ]:
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                               factor=0.2,
                                               patience=2,
                                               verbose=1,
                                               min_lr=1e-7)

In [ ]:
loaded_gs_model.compile(loss='sparse_categorical_crossentropy',
                        optimizer=tf.keras.optimizers.Adam(0.001),
                        metrics=['accuracy'])
history_101_food_classes_all_data_fine_tune = loaded_gs_model.fit(train_data,
                                                        epochs=100, # fine-tune for a maximum of 100 epochs
                                                        steps_per_epoch=len(train_data),
                                                        validation_data=test_data,
                                                        validation_steps=int(0.15 * len(test_data)), # validation during training on 15% of test data
                                                        callbacks=[create_tensorboard_callback("training_logs", "efficientb0_101_classes_all_data_fine_tuning"), # track the model training logs
                                                                   model_checkpoint, # save only the best model during training
                                                                   early_stopping, # stop model after X epochs of no improvements
                                                                   reduce_lr])

In [ ]:
loaded_gs_model.evaluate(test_data)


In [ ]:
loaded_gs_model.save("ANy path of your choice")

In [ ]:
!tensorboard dev upload --logdir ./ \
--name \
--description \
--one_shot

In [ ]:
!tensorboard dev list

In [ ]:
!tensorboard delete experiment_id "/"